# Problem 1 (Multiplying Polynomials) (40 Points)
In this coding assignment, the inputs will be polynomials. As done in class, a degree n polynomial p(x) = p0 + p1x + p2x2 + ··· + pnxn is represented as an array P[0 : n] where P[i] contains the coefficient pi. The input to the functions you write should be two lists P [0 : n] and Q[0 : n], and the output should be R[0 : 2n] which contains the coefficients of the polynomial r(x) = p(x)q(x).

### (a) Implement the naive multiplication (which takes O(n2) time). Recall, this just implements the formula for rk given as (1) in the Lecture 6 notes. (5 points)

In [56]:
import math
import time

# input P, Q are the two lists of polynomial coefficients
# output R, i.e., the product of the two polynomials
# given that P and Q are of the same length of [1:n], they must have the same order as well

def multi_poly(P, Q):
    # R contains the coefficients of the product polynomial
    R = []; n = len(P) - 1; val = 0
    # TODO -- base cases
    # for k <= 2n
    for k in range(0, 2 * n + 1):
        if (k <= n):
            val = 0
            # for i <= k
            for i in range(0, k + 1):
                val = val + (P[i] * Q[k-i])
        else:
            val = 0
            for i in range(k - n, n + 1):
                val = val + (P[i] * Q[k - i])
        R.append(val)
    return (R)

In [57]:
multi_poly([1,1,1],[2,3,1])

[2, 5, 6, 4, 1]

### (b) Implement Karatsuba’s algorithm done in class. If history is a guide, this is tricky to code mainly because you need to handle indices carefully. So be careful. (15 points)

In [54]:
''' 
input: Coefficients of two degree n polynomials: arrays P[0:n] 
        and Q[0:  n]
output: Coefficient of the product polynomial: array R[0: 2n].
size: n, the length of P and Q

'''
first_run = False

def karat_multi_poly(P, Q):
    global first_run
    n = (len(P) - 1); Px = []; Qx = []
    if (first_run == False):
        # initialization of arrays so we can set value to specific indice i and m
        first_run = True
        Px = [0 for i in range(len(P))]
        Qx = [0 for i in range(len(Q))]
    # base cases, 
    if len(P) == 0 or 1:
        # return R using naive multiplication
        return multi_poly(P, Q)
    # half and floor the array
    m = math.ceil(n/2)
    # loop through for all numbers 0 <= i <= m -1
    #initalize new arrays Px and Qx
    for i in range(0,m):
        Px[i] = P[i] + P[m + i]
        Qx[i] = Q[i] + Q[m + i]
    # if n = 2m
    if (n > (2 * m - 1)):
        Px[m] = P[n]
        Qx[m] = Q[n]
    #else:
        #do nothing as indices of Px and Qx are set to 0 from the start
         #Px[m] = 0
         #Qx[m] = 0
    # guarantees degree m, all have length <= 2m
    # array index range [0: 2(m-1)]
    # has coefficients of p1(x)* q1(x)
    R1 = karat_multi_poly(P[0:m-1], Q[0:m-1])
    # array index range [0: 2(n-m)]
    # has coefficients of p2(x)* q2(x)
    R2 = karat_multi_poly(P[m:n], Q[m:n])
    # array index range: [0: 2m]
    # has coefficients of (p1(x)+ p2(x)) * (q1(x)+ q2(x))
    R3 = karat_multi_poly(Px[0:m],Qx[0,m])
    # loop through all numbers 0 <= i <2m
    for i in range(0,(2*m + 1)):
        R4[i] = R3[i] - R1[i] - R2[i]
    # loop through all numbers 0 <= i <2n
    for i in range(2*(n + 1)):
        if (i >= (len(R1)) or i < 0):
            # because R1 = 0
            R[i] = R4[i-m]+ R2[i-2*m]
        if i >= ((len(R4)) or i < 0):
            R[i] = R1[i] + R2[i-2*m]
        if i >= ((len(R2)) or i < 0):
            R[i] = R1[i]+ R4[i-m]
        else:
            R[i] = R1[i]+ R4[i-m]+ R2[i-2*m]
        # assume an array returns 0 if indexed out of range
        # if i is less than range return 0
        # if i is greater than range return 0
    return R
        

In [55]:
karat_multi_poly([1,1,1],[2,3,1])

[2, 5, 6, 4, 1]

### (c) Given a number n, consider the polynomial pn(x) whose coefficient array Pn[0 : n] is defined as the first (n + 1)-digits of π. 
So, for instance,
p0(x)=3, p1(x)=3+x, p2(x)=3+x+4x2, p3(x)=3+x+4x2 +x3,...
Define rn(x) := pn(x) · pn(x), the square of the pn(x) polynomial. Let Sn be the sum of squares of the coefficients of the polynomial rn. So, for example, S0 = 81 (since r0(x) = p0(x)2 = 9 and so S0 =92),andS1 =118(sincer1(x)=p1(x)2 =9+6x+x2,and92 +62 +12 =118.)
Write code calling the Karatsuba routine to find out S20000? Note: we need to at the 20001 digits of π to answer this question. (5 points)
Even Karatsuba may take around a minute to run – don’t panic.

In [58]:
'''
helper function reading the n first numbers of pi and returns a list containing these,
the function was given on canvas
'''

def read_pi(n):
    #opens the file name "pi" and reads the first n digits
    #puts it in the list pi, and returns that list
    pi = list()
    f = open('pi','r')
    for i in range(n):
        d = f.read(1)
        pi.append(int(d))
    return pi

In [63]:
def square_pi_poly(n):
    # s is the sum of squares of the coefficients of the polynomial
    s = 0
    # read the first (n + 1) digits of pi
    P = read_pi(n)
    # TODO -- comment out
    # when we have karatsuba
    # call karatsuba's algorithm to get R, i.e.,
    # the square of the P(n) polynomial
    R = karat_multi_poly(P, P)
    for i in range(0, len(R)):
        s = s + math.pow(R[i],2)
    
    return s

In [64]:
print(square_pi_poly(3))

1062.0


### (d) In this part, you will find out at roughly what n does the Karatsuba algorithm “beat” the Naive algo- rithm. But we will measure “time” in two ways.

#### (i) First, we measure time using the time module of Python 3. Plot, as a function of n, n going from 1 to 10, 000 in jumps of 100, the time taken by both algorithms to compute rn(x). Show these two “curves” overlaid on the same plot. Clearly mark which is which. At roughly what n do you see Karatsuba “beating” the naive algorithm (if at all)? (5 points)

#### (ii) Second, we measure time as “arithmetic operations”. Every time you add, subtract, or multiply two entries from some array (either P or Q or something called recursively), you increment a global counter. This way, at the end of the code, you will know how many such operations have been performed. Note that we are not counting incrementing indices, or comparing, or anything else in the running time. Concretely, if you multiply two degree 1 polynomials, say (1 + x) and (2 + 3x), then there are 5 arithmetic operations by both: four multiplications and one addition of 2x and 3x.
Now plot, as a function of n, n going from 1 to 1000 in jumps of 5, the “time” (in this new notion) taken by both naive and Karatsuba algorithms. As before, show these two curves overlaid on the same plot. At roughly what n does Karatsuba beat Naive (if at all)? (5 points)
Is there a discrepancy in the two n’s you get above? Can you explain this?

#### (iii) Write code which takes input a number C > 0, and returns a plot where the x-axis has integers 1 to 200 in increments of 1, and which overlays Karatsuba’s time in terms of number of arithmetic operations (as in part (ii)) with the curve y(n) := C · nlog2 3. For what value of C, to the nearest integer, do you see the two curves “matching” (if at all)? That should give an idea of how big a constant is big-Oh hiding. (5 points)